## 0) Procedimento de alinhamento

In [1]:
import docker

a = %pwd
wd = (a.rsplit('/',1))[0]

REF_SEQ='Sars_cov_2.ASM985889v3.dna.toplevel.fa'
SEQ_IDS={'ERR4157959': 'portuguese', 'ERR4395294': 'spanish'}
SEQ = " ".join(SEQ_IDS.keys())

D_BWA='biocontainers/bwa:v0.7.17-3-deb_cv1'
D_SAMTOOLS='biocontainers/samtools:v1.7.0_cv4'
D_BCFTOOLS='quay.io/biocontainers/bcftools:1.9--h68d8f2e_7'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [ ]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_BWA,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/mutants',
    command='{0}/scripts_bash/bwa.sh {1} "{2}"'.format(WORK_CONT_DIRECTORY, REF_SEQ, SEQ))


In [28]:
sam_docker_cont =client.containers.run(
    image=D_SAMTOOLS,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/mutants',
    command='{0}/scripts_bash/sam.sh {1} "{2}"'.format(WORK_CONT_DIRECTORY, REF_SEQ, SEQ))

In [29]:
sam_docker_cont =client.containers.run(
    image=D_BCFTOOLS,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/mutants',
    command='{0}/scripts_bash/bcf.sh {1} "{2}"'.format(WORK_CONT_DIRECTORY, REF_SEQ, SEQ))

KeyboardInterrupt: 

## 1) Identificar a localização do gene na sequência de referência

In [2]:
import pprint
from BCBio import GFF

in_file = "../data/mutants/Sars_cov_2.ASM985889v3.101.gtf"

examiner = GFF.GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('MN908947.3',): 72},
 'gff_source': {('ensembl',): 72},
 'gff_source_type': {('ensembl', 'CDS'): 12,
                     ('ensembl', 'exon'): 12,
                     ('ensembl', 'gene'): 12,
                     ('ensembl', 'start_codon'): 12,
                     ('ensembl', 'stop_codon'): 12,
                     ('ensembl', 'transcript'): 12},
 'gff_type': {('CDS',): 12,
              ('exon',): 12,
              ('gene',): 12,
              ('start_codon',): 12,
              ('stop_codon',): 12,
              ('transcript',): 12}}


In [3]:
in_handle = open(in_file)
limit_info = dict(gff_id=["MN908947.3"], gff_type=["gene"])
for rec in GFF.parse(in_handle, limit_info=limit_info):
    for gene in rec.features:
        if 'ENSSASG00005000009' in gene.qualifiers['gene_id']:
            ORF7a_ref = gene
in_handle.close()

## 2) Identificar mutações no gene nas amostraras

In [4]:
from scripts_python.vcf_gff import vcf2dict, lookup_var2genes

samples_vars = {}
samples_vars_ORF7a = {}

for sample in SEQ_IDS.keys():
    sample_variations = vcf2dict("{0}/data/mutants/{1}/{1}.sars_cov.raw.vcf".format(wd,sample))
    samples_vars[sample] = sample_variations
    # filter to var in location
    for var in sample_variations.keys():
        samples_vars_ORF7a[sample] = lookup_var2genes(sample_variations, ORF7a_ref)


# Ideas
https://covid-19.ensembl.org/info/docs/tools/vep/index.html
